# Topological Index Calculator
This notebook calculates different topological indices (Edge Density, Wiener Index, Petitjean Index) from molecular structures.

In [1]:
import os
import sys
import requests
from urllib.parse import urlparse


### Function 1: get_index_choice
It ask the user: **"USER! KINDLY GIVE ME AN INDEX TO CALCULATE"** 

In [2]:
def get_index_choice():
    while True:
        print('Which topological index would you like to calculate?')
        print('1 - Edge Density')
        print('2 - Wiener Index')
        print('3 - Petitjean Index')
        print('4 - All of the above')

        choice = input('Enter your choice (1-4): ').strip()

        options = {
            '1': 'edge',
            '2': 'wiener',
            '3': 'petitjean',
            '4': 'all'
        }

        selected = options.get(choice)
        if selected:
            return selected
        print('Invalid choice. Please select 1, 2, 3, or 4.')


### Function 2: get_input_path
This function is responsible for accepting the path to a file or folder from the user. It checks if the file or folder exists, ensuring that the program does not attempt to process non-existing paths. This function is essential for proper file handling in the script, especially when the input could be either a single file or a directory containing multiple files.

In [3]:
def get_input_path():
    input_path = input('Enter the path to a .mol file, a folder of .mol files, or a .sdf file, or a URL: ').strip()

    if input_path.startswith('http'):
        return download_from_url(input_path)
    elif os.path.exists(input_path):
        return input_path
    else:
        print('The specified path does not exist.')
        sys.exit(1)


### Function 3: download_from_url
This function is essential for downloading files from URLs provided by the user. If the user provides a URL instead of a local path, the function will handle downloading the file and save it to the local machine. This function is important for ensuring that remote files can be processed by the script, which extends the utility of the tool beyond just local files.

In [4]:
def download_from_url(url):
    """
    Downloads a file from the given URL and returns the file path.
    If the URL is invalid or file cannot be downloaded, raises an error.
    """
    try:
        filename = os.path.basename(urlparse(url).path)
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                file.write(response.content)
            print(f'File downloaded: {filename}')
            return filename
        else:
            print(f'Failed to download file. Status code: {response.status_code}')
            sys.exit(1)
    except Exception as e:
        print(f'Error downloading the file: {e}')
        sys.exit(1)


### Function 4: detect_input_type
This function is responsible for identifying the type of input (whether it's a file, folder, or URL) so the appropriate processing steps can be followed. It is an important step to ensure that the program can correctly handle various input formats (e.g., single files, folders, or multiple structures in .sdf format). This enables the flexibility of the tool to process diverse molecular data sources.

In [5]:
def detect_input_type(input_path):
    if os.path.isdir(input_path):
        return 'mol_folder'
    elif input_path.endswith('MOL:
        return 'mol_file'
    elif input_path.endswith('SDF'):
        return 'sdf_file'
    else:
        print('Unsupported file type. Please provide a .mol file, folder, or .sdf file.')
        sys.exit(1)


### Example Usage
The following code demonstrates how to use the functions to select an index, input a file or folder, and determine the file type. It also shows how to combine all functions to compute the selected topological index.

In [6]:
# Example usage:
index_choice = get_index_choice()
input_path = get_input_path()
input_type = detect_input_type(input_path)

print(f'Selected index: {index_choice}')
print(f'Input path: {input_path}')
print(f'Input type: {input_type}')